In [89]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from facenet_pytorch import MTCNN, InceptionResnetV1
from PIL import Image

# Define the dataset class
class FaceVerificationDataset(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.image_paths = self.get_image_paths()

    def get_image_paths(self):
        image_paths = []
        for root, dirs, files in os.walk(self.root_dir):
            for file in files:
                if file.endswith(".jpg") or file.endswith(".png"):
                    image_paths.append(os.path.join(root, file))
        return image_paths

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        return image

# Set the root directory of your dataset
root_dir = "C:\\Users\\gojas\\Downloads\\val"

# Create an instance of the dataset
dataset = FaceVerificationDataset(root_dir)

# Create MTCNN face detector for face detection and alignment
mtcnn = MTCNN()

# Create InceptionResnetV1 model for face recognition
model = InceptionResnetV1(pretrained='vggface2').eval()

# Custom collate function to convert PIL Images to tensors
def custom_collate_fn(batch):
    batch_images = []
    for image in batch:
        image = image.resize((160, 160))  # Resize the image to a consistent size
        image = transforms.ToTensor()(image)
        batch_images.append(image)
    return torch.stack(batch_images)

# Create a data loader for the dataset with the custom collate function
data_loader = DataLoader(dataset, batch_size=8, shuffle=False, collate_fn=custom_collate_fn)

# Specify the device to be used (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the specified device
model = model.to(device)

# Extract embeddings for face verification
embeddings = []
for images in data_loader:
    with torch.no_grad():
        # Preprocess images and detect faces
        aligned_images = images.to(device)

        # Compute embeddings
        batch_embeddings = model(aligned_images)
        embeddings.extend(batch_embeddings.detach().cpu().numpy())
    torch.cuda.empty_cache()

In [86]:
def compare_embeddings(embedding1, embedding2):
    # Compute L2 distance between embeddings
    distance = np.linalg.norm(embedding1 - embedding2)
    return distance

In [87]:
labels_file = "C:\\Users\\gojas\\Downloads\\val.txt"
# Load the labels from the text file
labels = []
with open(labels_file, 'r') as f:
    lines = f.readlines()[1:]  # Skip the header line
    for line in lines:
        _, label = line.strip().split(',')
        label = int(label.strip())
        labels.append(label)
labels = np.array(labels)

100


In [103]:
embeddings_array = np.array(embeddings)
predicted_labels = []

for i in range(0, len(embeddings_array), 2):
    condition = compare_embeddings(embeddings_array[i], embeddings_array[i+1]) < 1
    if np.all(condition):
        predicted_labels.append(1)  
    else:
        predicted_labels.append(0)  


# Calculate accuracy
accuracy = np.mean(predicted_labels == labels)
print(f"Accuracy: {accuracy}")




Accuracy: 0.96


In [105]:
with open('output.txt', 'w') as f:
    # Write each element of the output_data list to a new line in the file
    for item in predicted_labels:
        f.write(str(item) + '\n')